In [12]:
import numpy as np
import pandas as pd
import lightgbm as lgb
path='/home/student303/oyx/almama/'
data = pd.read_hdf(path+ 'somefeature/base_feature_all_0_B.h5', key = 'a')

## 用户上次点击同一商铺的时间间隔

In [14]:
%%time
#feature1 用户上次点击同一商铺时间间隔
data=data.sort_values(by=['context_timestamp'],ascending=True)
d={}
def user_lastclicktime_interval(x):
    newtime=x.context_timestamp
    name=str(x.user_id)+'_'+str(x.shop_id)
    if name not in d:
        d[name]=newtime
        return -1
    time_interval=d[name]-newtime
    d[name]=newtime
    return time_interval
data['user_lastclick_sameshop_interval']=data.apply(user_lastclicktime_interval,axis=1)

data=data.sort_values(by=['context_timestamp'],ascending=False)
#feature2 用户下次点击同一商铺时间间隔(当天)
d={}
def user_lastclicktime_interval(x):
    newtime=x.context_timestamp
    name=str(x.user_id)+'_'+str(x.shop_id)+'_'+str(x.day)
    if name not in d:
        d[name]=newtime
        return -1
    time_interval=d[name]-newtime
    d[name]=newtime
    return time_interval
data['user_nextclick_sameshop_interval']=data.apply(user_lastclicktime_interval,axis=1)

CPU times: user 46.6 s, sys: 680 ms, total: 47.3 s
Wall time: 47.3 s


## 用户上次点击的时间间隔

In [15]:
%%time
#feature3 用户上次点击时间间隔
data=data.sort_values(by=['context_timestamp'],ascending=True)
d={}
def user_nextclicktime_interval(x):
    newtime=x.context_timestamp
    name=str(x.user_id)
    if name not in d:
        d[name]=newtime
        return -1
    time_interval=d[name]-newtime
    d[name]=newtime
    return time_interval
data['user_lastclick_interval']=data.apply(user_nextclicktime_interval,axis=1)

#feature4 用户下次点击时间间隔(当天)
data=data.sort_values(by=['context_timestamp'],ascending=False)
d={}
def user_nextclicktime_interval(x):
    newtime=x.context_timestamp
    name=str(x.user_id)+'_'+str(x.day)
    if name not in d:
        d[name]=newtime
        return -1
    time_interval=d[name]-newtime
    d[name]=newtime
    return time_interval
data['user_nextclick_interval']=data.apply(user_nextclicktime_interval,axis=1)

CPU times: user 34.9 s, sys: 556 ms, total: 35.5 s
Wall time: 35.5 s


## 分类器验证

In [18]:
#Define featurelist
droplist=['instance_id',
          'item_category_list',
          'item_property_list',
          'user_id', 
          'context_id',
          'context_timestamp',
          'predict_category_property',
          'is_trade',
          'day', 'min', 'second','subm']
featurelist=[i for i in data.columns if i not in droplist]

In [19]:
featurelist

['item_id',
 'item_brand_id',
 'item_city_id',
 'item_price_level',
 'item_sales_level',
 'item_collected_level',
 'item_pv_level',
 'user_gender_id',
 'user_age_level',
 'user_occupation_id',
 'user_star_level',
 'context_page_id',
 'shop_id',
 'shop_review_num_level',
 'shop_review_positive_rate',
 'shop_star_level',
 'shop_score_service',
 'shop_score_delivery',
 'shop_score_description',
 'item_category_hash',
 'hour',
 'user_lastclick_sameshop_interval',
 'user_nextclick_sameshop_interval',
 'user_lastclick_interval',
 'user_nextclick_interval']

In [20]:
print('LGBM Training, Feature dim : ',len(featurelist))

params = {
    'task': 'train',
    'objective': 'binary',
    'metric': 'binary_logloss',
    'max_depth':4,
    #'num_leaves': 6,
    #'min_data_in_leaf':1,
    'learning_rate': 0.1,
    'feature_fraction': 0.8,
    'bagging_fraction': 1.0,
    'verbose': 0,
}

d_train=lgb.Dataset(data.loc[data.day < 24][featurelist], label=data.loc[data.day < 24]['is_trade'])
d_valid=lgb.Dataset(data.loc[data.day == 24][featurelist], label=data.loc[data.day == 24]['is_trade'])
model = lgb.train(params, train_set=d_train, num_boost_round=600, valid_sets=[d_train,d_valid], \
              early_stopping_rounds=50, verbose_eval=10)

LGBM Training, Feature dim :  25
Train until valid scores didn't improve in 50 rounds.
[10]	training's binary_logloss: 0.238875	valid_1's binary_logloss: 0.235753
[20]	training's binary_logloss: 0.129254	valid_1's binary_logloss: 0.12416
[30]	training's binary_logloss: 0.0987823	valid_1's binary_logloss: 0.0923993
[40]	training's binary_logloss: 0.0907006	valid_1's binary_logloss: 0.0837399
[50]	training's binary_logloss: 0.0884302	valid_1's binary_logloss: 0.0813257
[60]	training's binary_logloss: 0.0874839	valid_1's binary_logloss: 0.0805069
[70]	training's binary_logloss: 0.0868656	valid_1's binary_logloss: 0.0801385
[80]	training's binary_logloss: 0.086404	valid_1's binary_logloss: 0.0799592
[90]	training's binary_logloss: 0.0860389	valid_1's binary_logloss: 0.0798161
[100]	training's binary_logloss: 0.0857254	valid_1's binary_logloss: 0.0796975
[110]	training's binary_logloss: 0.0854227	valid_1's binary_logloss: 0.0796467
[120]	training's binary_logloss: 0.0851727	valid_1's binary

In [23]:
data.to_hdf(path + 'somefeature/419timefeature.h5', key='a')